In [2]:
!pip install opencv-python scikit-learn numpy

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.3/39.5 MB ? eta -:--:--
    --------------------------------------- 0.5/39.5 MB 1.4 MB/s eta 0:00:28
    --------------------------------------- 0.8/39.5 MB 1.4 MB/s eta 0:00:28
   - -------------------------------------- 1.0/39.5 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.0/39.5 MB 1.3 MB/s eta 0:00:30
   - -------------------------------------- 1.6/39.5 MB 1.3 MB/s eta 0:00:31
   - -------------------------------------- 1.8/39.5 MB 1.3 MB/s eta 0:00:30
   -- ------------------------------------- 2.1/39.5 MB 1.4 MB/s eta 0:00:28
   -- ------------------------------------- 2.4/39.5 MB 1.4 MB/s eta 0:00:27
   -- ------------------------------------- 2.9/39.5 MB 1.4 MB/s eta 0:00:26
   --- ------------

In [2]:
# Import required libraries
import os
import cv2
import numpy as np
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

In [3]:
# Define a function to load and preprocess images from a given folder
def load_images_from_folder(base_path, image_size=(128 , 128)):
    images = []
    labels = []

    for class_name in sorted(os.listdir(base_path)):
        class_path = os.path.join(base_path, class_name)
        if not os.path.isdir(class_path):
            continue

        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            img = cv2.imread(img_path)

            if img is not None:
                img = cv2.resize(img, image_size)
                images.append(img.flatten())
                labels.append(class_name)

    return np.array(images), np.array(labels)

In [10]:
# Main function to train and evaluate the SVM classifier
def run_image_classifier(train_folder, test_folder):
    print("Loading training data...")
    X_train, y_train = load_images_from_folder(train_folder)
    print(f"Loaded {len(X_train)} training images.")

    print("Loading test data...")
    X_test, y_test = load_images_from_folder(test_folder)
    print(f"Loaded {len(X_test)} test images.")

    label_encoder = LabelEncoder()
    y_train_encoded = label_encoder.fit_transform(y_train)
    y_test_encoded = label_encoder.transform(y_test)

    print("Training SVM classifier...")
    model = make_pipeline(StandardScaler(), SVC(kernel='sigmoid', C=1.0 , random_state=42))
    model.fit(X_train, y_train_encoded)

    print("Evaluating model...")
    y_pred = model.predict(X_test)
    print(classification_report(y_test_encoded, y_pred, target_names=label_encoder.classes_))

In [12]:
# Provide paths to your training and test data directories
train_folder = r"D:\Animals Dataset\Animals Dataset\Train Data"  # e.g., "D:\Animals Dataset\Animals Dataset\Train Data"    D:\cifar10-raw-images\images\train
test_folder = r"D:\Animals Dataset\Animals Dataset\Test Data"    # e.g., "D:\Animals Dataset\Animals Dataset\Test Data"    D:\cifar10-raw-images\images\test

# Run the classifier
run_image_classifier(train_folder, test_folder)

Loading training data...
Loaded 2846 training images.
Loading test data...
Loaded 153 test images.
Training SVM classifier...
Evaluating model...
              precision    recall  f1-score   support

        cats       0.42      0.49      0.45        51
        dogs       0.51      0.41      0.46        51
       panda       0.65      0.67      0.66        51

    accuracy                           0.52       153
   macro avg       0.53      0.52      0.52       153
weighted avg       0.53      0.52      0.52       153

